In [ ]:
import wandb
wandb.login(key='24434976526d9265fdbe2b2150787f46522f5da4')

import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
import shutil
import os
import random
import pytorch_lightning as pl
from types import SimpleNamespace

from pytorch_lightning.loggers import WandbLogger
%matplotlib inline

prefix='/kaggle/input/inaturalist12k/Data/inaturalist_12K/'

data_prefix='/kaggle/working/'
classes=['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

    
flag=os.path.exists(data_prefix+'splittedVal')



valid_split=0.2
if not flag:
    for each in ['train','val']:
        shutil.copytree(prefix+each,data_prefix+each)
    os.mkdir(data_prefix+"splittedVal")
    for each in classes:
        images = os.listdir(data_prefix+'train/'+each+'/')
        random.shuffle(images)
        valid_till=int(len(images)*valid_split)
        os.mkdir(data_prefix+'splittedVal/'+each)
        for i in range(valid_till):
            shutil.move(data_prefix+'train/'+each+"/"+images[i],data_prefix+'splittedVal/'+each)
            



In [ ]:
def data_distribution(dataset):
    
    print("Total number of samples :",len(dataset.targets))
    print("Number of classes :",len(set(dataset.targets)),'\n')
    idx_to_class = {v:k for k, v in dataset.class_to_idx.items()}
    
    idx_distribution = dict(Counter(dataset.targets))
    
    for k,v in idx_distribution.items():
        print(idx_to_class[k], " : ",v)

transform = transforms.Compose([
                        transforms.Resize((224, 224)),
                        transforms.ToTensor(),
                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
                        ])

train_dataset = torchvision.datasets.ImageFolder(root=data_prefix+'train', transform=transform)
print("Train data...\n")
data_distribution(train_dataset)

train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size
valid_dataset = torchvision.datasets.ImageFolder(root=data_prefix+'splittedVal', transform=transform)
print("Valid data...\n")
data_distribution(valid_dataset)

test_dataset = torchvision.datasets.ImageFolder(root=data_prefix+'val', transform=transform)
print("Test data...\n")
data_distribution(test_dataset)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

valid_loader=DataLoader(valid_dataset, batch_size=16, shuffle=False)

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

idx_to_class = {v:k for k, v in train_dataset.class_to_idx.items()}

xx,yy=next(iter(train_loader))
fig=plt.figure(figsize=(25,10))
for i in range(1,11):
    ax=fig.add_subplot(2,5,i)
    x=xx[i]
    y=yy[i]
    x=x.permute(1,2,0)
    ax.imshow(x,cmap='gray')
    ax.set_title(idx_to_class[y.item()])

In [ ]:
def getActivation(function):
    if function=='ReLU':
        return nn.ReLU()
    if function=='GELU':
        return nn.GELU()
    if function=='SiLU':
        return nn.SELU()
    if function=='Mish':
        return nn.Mish()
    return nn.ReLU() # if no match
    
    

class Model(pl.LightningModule):
    def __init__(self,config):
        
        super().__init__()
        self.learning_rate=config.learning_rate
        layers=[]
        input_channels=3
        num_layers=5
        kernel_size=3
        kernel_stride=1
        max_pool_size=2
        max_pool_stride=2
        filters=[]
        if(config.filter_organization=='same'):
            filters=[config.filter_size]*num_layers
        elif(config.filter_organization=='double'):
            filters.append(config.filter_size)
            for i in range(4):
                filters.append(filters[-1]*2)
        elif(config.filter_organization=='halve'):
            filters.append(config.filter_size)
            for i in range(4):
                filters.append(filters[-1]//2)

        filters.insert(0,input_channels)
        out_height=224
        for i in range(5):
            layers.append(nn.Conv2d(filters[i],filters[i+1],kernel_size = kernel_size))
            out_height=(out_height-kernel_size)//kernel_stride+1
            layers.append(nn.MaxPool2d(kernel_size = max_pool_size,stride = max_pool_stride))  
            out_height=out_height//max_pool_stride
            layers.append(getActivation(config.activation))
            if(config.batch_normalisation=='Yes'):
                layers.append(nn.BatchNorm2d(filters[i+1]))
        layers.append(nn.Flatten())
        layers.append(nn.Dropout(config.dropout))
        layers.append(nn.Linear(out_height*out_height*filters[-1],config.dense_layer_size))
        layers.append(nn.Linear(config.dense_layer_size,10))
        self.net = nn.Sequential(*layers)
        self.loss = nn.CrossEntropyLoss()
        self.valid_loss=[]
        self.valid_acc=[]
        self.train_loss=[]
        self.train_acc=[]
        
  
        
    def forward(self,x):
        return self.net(x)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr= self.learning_rate)

    def training_step(self,batch,batch_idx):
        X,Y = batch
        output = self(X)
        loss = self.loss(output,Y)
        acc = (output.argmax(dim = 1) == Y).float().mean()
        self.train_loss.append(loss)
        self.train_acc.append(acc)
        return loss


    def validation_step(self, batch,batch_idx):
        X,Y = batch
        output = self(X)
        loss = self.loss(output,Y)
        acc = (output.argmax(dim = 1) == Y).float().mean()
        self.valid_loss.append(loss)
        self.valid_acc.append(acc)
        return loss
    def on_train_epoch_end(self):
        valid_loss=sum(self.valid_loss)/len(self.valid_loss)
        valid_acc=sum(self.valid_acc)/len(self.valid_acc)
        train_loss=sum(self.train_loss)/len(self.train_loss)
        train_acc=sum(self.train_acc)/len(self.train_acc)
        self.train_acc=[]
        self.train_loss=[]
        self.valid_loss=[]
        self.valid_acc=[]
        print(f"Epoch: {self.current_epoch} train accuracy :{train_acc:.2f} valid_accuracy :{valid_acc:.2f}")
        

        wandb.log({'train_acc':train_acc,'train_loss':train_loss,'valid_acc':valid_acc,'valid_loss':valid_loss})
  
  

config= {
    'method': 'bayes',
    'name': 'CNN Assign 2',
    'metric': {
        'goal': 'maximize', 
        'name': 'valid_acc'
      },
    "parameters":
    {
    "filter_size":{
      "values" :[32,64]
    },
    "data_augumentation" :{
        "values" : ["Yes","No"]
    },
    "batch_normalisation" :{
        "values" : ["Yes","No"]
    },
    "dropout" :{
        "values" : [0,0.1]
    },
    
     "filter_organization" :{
        "values" : ['same','halve','double']
    },
    
    "activation" :{
          "values" : ["ReLU","GELU","SiLU","Mish"]
    },
    "epochs" :{
          "values" : [10]
    },
    "dense_layer_size" :{
          "values" : [256,128]
    },
      "learning_rate" :{
          "values" : [0.0001]
    }
    
  }

}


def main():
    with wandb.init(save_code=False) as run:
        params=dict(wandb.config)
        params=SimpleNamespace(**params)
        run_name=f'FZ-{params.filter_size} AF - {params.activation} filter_org- {params.filter_organization} batch_norm -{params.batch_normalisation} data_aug -{params.data_augumentation} dropout- {params.dropout}'
        wandb.run.name=run_name
        if(params.data_augumentation=='Yes'):
            transform = transforms.Compose([
                        transforms.Resize((224, 224)),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomRotation(10),
                       ])
        else:
            transform = transforms.Compose([
                        transforms.Resize((224, 224)),
                        transforms.ToTensor(),
                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
                        ])
            
        
        train_dataset = torchvision.datasets.ImageFolder(root=data_prefix+'train', transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
        model = Model(params) 
        trainer = pl.Trainer(max_epochs=params.epochs,devices=1,accelerator="gpu") 
        trainer.fit(model,train_loader,valid_loader)

sweep_id = wandb.sweep(sweep=config, project='CNN_V2')

wandb.agent(sweep_id, main, count=40)
wandb.finish()